In [1]:
import os
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import optim
from torchvision.datasets import MNIST
from torchvision.utils import save_image
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")

In [2]:
if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')

number_epochs = 10
batch_size = 128
learning_rate = 1e-4

transform_image = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = MNIST('./data', transform=transform_image)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

def to_image(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

In [6]:
class VariationalAutoEncoder(nn.Module):
    def __init__(self):
        super(VariationalAutoEncoder, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode_function(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode_function(self, z):
        h3 = F.relu(self.fc3(z))
        return F.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode_function(x)
        z = self.reparametrize(mu, logvar)
        return self.decode_function(z), mu, logvar


model = VariationalAutoEncoder()
if torch.cuda.is_available():
    model.cuda()

reconstruction_function = nn.MSELoss(size_average=False)


def loss_function(reconstruction_x, x, mu, latent_log_variance):
    """
    reconstruction_x: generating images
    x: original images
    mu: latent mean
    """
    BCE = reconstruction_function(reconstruction_x, x)  
    # KL loss = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_aspect = mu.pow(2).add_(latent_log_variance.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_aspect).mul_(-0.5)
    # KL divergence
    return BCE + KLD


optimizer = optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(number_epochs):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(data_loader):
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img)
        if torch.cuda.is_available():
            img = img.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(img)
        loss = loss_function(recon_batch, img, mu, logvar)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(img),
                len(data_loader.dataset), 100. * batch_idx / len(data_loader),
                loss.data[0] / len(img)))

    print('Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(data_loader.dataset)))
    if epoch % 10 == 0:
        save = to_image(recon_batch.cpu().data)
        save_image(save, './vae_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './vae.pth')

Train Epoch: 0 [0/60000 (0%)]	Loss: 1507.728882
Train Epoch: 0 [12800/60000 (21%)]	Loss: 832.425720
Train Epoch: 0 [25600/60000 (43%)]	Loss: 777.984314
Train Epoch: 0 [38400/60000 (64%)]	Loss: 761.669189
Train Epoch: 0 [51200/60000 (85%)]	Loss: 747.355591
Epoch: 0 Average loss: 831.2576
Train Epoch: 1 [0/60000 (0%)]	Loss: 747.032776
Train Epoch: 1 [12800/60000 (21%)]	Loss: 740.493469
Train Epoch: 1 [25600/60000 (43%)]	Loss: 739.800781
Train Epoch: 1 [38400/60000 (64%)]	Loss: 738.366272
Train Epoch: 1 [51200/60000 (85%)]	Loss: 735.688477
Epoch: 1 Average loss: 739.2957
Train Epoch: 2 [0/60000 (0%)]	Loss: 735.274780
Train Epoch: 2 [12800/60000 (21%)]	Loss: 735.723389
Train Epoch: 2 [25600/60000 (43%)]	Loss: 733.795105
Train Epoch: 2 [38400/60000 (64%)]	Loss: 730.536743
Train Epoch: 2 [51200/60000 (85%)]	Loss: 732.716736
Epoch: 2 Average loss: 733.2621
Train Epoch: 3 [0/60000 (0%)]	Loss: 730.552795
Train Epoch: 3 [12800/60000 (21%)]	Loss: 731.892944
Train Epoch: 3 [25600/60000 (43%)]	Loss